In [ ]:
import os 
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.",
        metadata={"source": "us-presidents-doc"},
    ),
    Document(
        page_content="Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.",
        metadata={"source": "us-politics-doc"},
    ),
    Document(
        page_content="The Kennedy family is known for their significant influence in American politics and their extensive philanthropic efforts.",
        metadata={"source": "kennedy-family-doc"},
    ),
    Document(
        page_content="Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.",
        metadata={"source": "us-senators-doc"},
    ),
    Document(
        page_content="Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.",
        metadata={"source": "first-lady-doc"},
    ),
]

### Vector Stores

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding = OpenAIEmbeddings()
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


### Similarity Search

In [3]:
vectorstore.similarity_search("John")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
 Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.'),
 Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
 Document(metadata={'source': 'first-lady-doc'}, page_content='Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.')]

In [4]:
vectorstore.similarity_search_with_score("John")

[(Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
  0.45017221570014954),
 (Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.'),
  0.46441298723220825),
 (Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
  0.47508904337882996),
 (Document(metadata={'source': 'first-lady-doc'}, page_content='Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.'),
  0.48099976778030396)]

### Retrievers

In [6]:
from typing import List 
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

retriever.batch(["John","Robert"])

[[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.')],
 [Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.')]]

In [7]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)

retriever.batch(["John","Robert"])

[[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.')],
 [Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.')]]

### Retrievers are Runnables

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """ 
 Answer this question using the provided context only.

 {question}

 Context:
 {context}

"""

prompt = ChatPromptTemplate.from_messages([("human",message)])

chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | llm 

In [10]:
response = chain.invoke("Tell me about the Jackie")
print(response.content)

The Jackie refers to Jacqueline Kennedy Onassis, the wife of John F. Kennedy and an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.
